tiktoken 是 OpenAI 开源的快速 Tokenizer 工具，用于将文本转换为大模型所需的 token。

In [1]:
import tiktoken

In [2]:
tiktoken.list_encoding_names()  # 查看所有可用的encoding

# gpt2的结构与其他不同，下文将跳过对gpt2的缓存，如有需要，请自行研究

['gpt2', 'r50k_base', 'p50k_base', 'p50k_edit', 'cl100k_base', 'o200k_base']

In [3]:
from tiktoken.registry import ENCODINGS

ENCODINGS.keys()

# 内存缓存，默认为空，加载之后会填充

dict_keys([])

In [4]:
# 如果没有设置缓存目录，会触发下载

encoding = tiktoken.get_encoding("o200k_base")
encoding.encode("请帮我生成一份关于Deep Research的报告")

[10394, 58626, 7522, 70033, 2432, 26180, 51566, 46422, 11086, 1616, 65247]

In [5]:
ENCODINGS.keys()

dict_keys(['o200k_base'])

# 离线方案

在 .venv\Lib\site-packages\tiktoken_ext\openai_public.py 可以看到下载地址

## ~~直接使用本地路径~~

这条路行不通

~~最简单、方便，如果是自己直接使用，推荐。如果是使用了第三方的sdk，可能需要修改源代码~~

In [6]:
encoding = tiktoken.get_encoding("D:/AppData/tiktoken/encodings/o200k_base.tiktoken")
encoding.encode("请帮我生成一份关于Deep Research的报告")

ValueError: Unknown encoding D:/AppData/tiktoken/encodings/o200k_base.tiktoken.
Plugins found: ['tiktoken_ext.openai_public']
tiktoken version: 0.9.0 (are you on latest?)

## 设置本地缓存

只能使用这个方案，除非我们不使用tiktoken的encoding，比如我们使用transformers库的AutoTokenizer。

如果使用了第三方的sdk，三方sdk使用tiktoken且没有预留可配置的接口，我们就需要使用这个方案了（除非我们修改第三方sdk的代码）

根据 tiktoken/load.py 文件 read_file_cached方法，需要先将文件重命名为hash值，根据文件的下载地址进行hash

In [7]:
import hashlib

download_url = {
    'r50k_base': 'https://openaipublic.blob.core.windows.net/encodings/r50k_base.tiktoken',
    'p50k_base': 'https://openaipublic.blob.core.windows.net/encodings/p50k_base.tiktoken',
    'cl100k_base': 'https://openaipublic.blob.core.windows.net/encodings/cl100k_base.tiktoken',
    'o200k_base': 'https://openaipublic.blob.core.windows.net/encodings/o200k_base.tiktoken',
}

def cache_key(name: str):
    return hashlib.sha1(download_url[name].encode()).hexdigest()

In [8]:
cache_key_dict = {key: (value, cache_key(key)) for key, value in download_url.items()}
cache_key_dict

{'r50k_base': ('https://openaipublic.blob.core.windows.net/encodings/r50k_base.tiktoken',
  '0ea1e91bbb3a60f729a8dc8f777fd2fc07cd8df4'),
 'p50k_base': ('https://openaipublic.blob.core.windows.net/encodings/p50k_base.tiktoken',
  'ec7223a39ce59f226a68acc30dc1af2788490e15'),
 'cl100k_base': ('https://openaipublic.blob.core.windows.net/encodings/cl100k_base.tiktoken',
  '9b5ad71b2ce5302211f9c61530b329a4922fc6a4'),
 'o200k_base': ('https://openaipublic.blob.core.windows.net/encodings/o200k_base.tiktoken',
  'fb374d419588a4632f3f557e76b4b70aebbca790')}

In [9]:
import os

os.environ['TIKTOKEN_CACHE_DIR'] = 'D:/AppData/tiktoken/encodings/cache'

In [10]:
text = '请帮我生成一份关于Deep Research的报告'
{key : str(tiktoken.get_encoding(key).encode(text)) for key in download_url.keys()}

{'r50k_base': '[46237, 115, 30585, 106, 22755, 239, 37955, 22755, 238, 31660, 20015, 121, 17739, 111, 12859, 236, 29744, 4992, 21410, 162, 232, 98, 37772, 232]',
 'p50k_base': '[46237, 115, 30585, 106, 22755, 239, 37955, 22755, 238, 31660, 20015, 121, 17739, 111, 12859, 236, 29744, 4992, 21410, 162, 232, 98, 37772, 232]',
 'cl100k_base': '[15225, 13821, 106, 37046, 45059, 15120, 70542, 30356, 35304, 34564, 8483, 9554, 43378, 58655]',
 'o200k_base': '[10394, 58626, 7522, 70033, 2432, 26180, 51566, 46422, 11086, 1616, 65247]'}

# 平替方案：使用transformers

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("D:/Models/BAAI/bge-large-zh-v1.5")
tokenizer.encode(text)

d:\Workspace\Projects\2025\gpt-researcher\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[101,
 6435,
 2376,
 2769,
 4495,
 2768,
 671,
 819,
 1068,
 754,
 12592,
 10018,
 4638,
 2845,
 1440,
 102]